In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

В начале обучим простую линейную модель.

In [2]:
stars = pd.read_csv('./data/StarsClassification.csv')
stars

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
0,3068,0.002400,0.1700,16.12,0,Red,M
1,3042,0.000500,0.1542,16.60,0,Red,M
2,2600,0.000300,0.1020,18.70,0,Red,M
3,2800,0.000200,0.1600,16.65,0,Red,M
4,1939,0.000138,0.1030,20.06,0,Red,M
...,...,...,...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93,5,Blue,O
236,30839,834042.000000,1194.0000,-10.63,5,Blue,O
237,8829,537493.000000,1423.0000,-10.73,5,White,A
238,9235,404940.000000,1112.0000,-11.23,5,White,A


Значение столбцов:

Temperature (K) -- Абсолютная температура (в K)\
Luminosity (L/Lo) -- Относительная яркость (L/Lo)\
Radius (R/Ro) -- Относительный радиус (R/Ro)\
Absolute magnitude (Mv) -- Абсолютная величина (Mv)\
Star type -- Тип звезды **(Красный карлик, Коричневый карлик, Белый карлик, Главная последовательность, Сверхгиганты, Гипергиганты)**\
Star color -- Цвет звезды (белый, красный, синий, желтый, желто-оранжевый и т. д.) Спектральный класс (O,B,A,F,G,K,,M)\
Spectral Class -- Спектральный класс (O,B,A,F,G,K,,M)

Переведем множество Spectral Class в множество целых значений и удалим колонку Star color

In [3]:
classDict = {'O': 0,'B': 1,'A': 2,'F':3,'G':4,'K':5,'M':6}

stars['Spectral Class'] = stars['Spectral Class'].apply(lambda x: classDict[x])
stars = stars.drop(['Star color'], axis=1)
stars

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Spectral Class
0,3068,0.002400,0.1700,16.12,0,6
1,3042,0.000500,0.1542,16.60,0,6
2,2600,0.000300,0.1020,18.70,0,6
3,2800,0.000200,0.1600,16.65,0,6
4,1939,0.000138,0.1030,20.06,0,6
...,...,...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93,5,0
236,30839,834042.000000,1194.0000,-10.63,5,0
237,8829,537493.000000,1423.0000,-10.73,5,2
238,9235,404940.000000,1112.0000,-11.23,5,2


In [4]:
# Создадим датасет для использования в градиентном бустинге

stars['id'] = [i for i in range(len(stars))]
stars = stars.set_index('id')
stars_for_lgb = stars.copy()
stars_for_lgb

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Spectral Class
id,,,,,,
0,3068,0.002400,0.1700,16.12,0,6
1,3042,0.000500,0.1542,16.60,0,6
2,2600,0.000300,0.1020,18.70,0,6
3,2800,0.000200,0.1600,16.65,0,6
4,1939,0.000138,0.1030,20.06,0,6
...,...,...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93,5,0
236,30839,834042.000000,1194.0000,-10.63,5,0
237,8829,537493.000000,1423.0000,-10.73,5,2


Мы хотим предсказывать класс звезды, основываясь на значениях температуры, яркости, радуиуса и т.д.

In [5]:
X = stars.drop(['Spectral Class'], axis=1)
y = stars['Spectral Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model=LinearRegression()
model.fit(X_train, y_train)
for column, coef in zip(X_train.columns, model.coef_):
    print(column, coef)
print(model.intercept_)

Temperature (K) -0.0001824097703568863
Luminosity(L/Lo) -2.7230327589667844e-07
Radius(R/Ro) 0.002048864783124547
Absolute magnitude(Mv) -0.15590293604948466
Star type -1.5486826790445993
9.507246532504128


Ошибки предсказания:

In [6]:
y_pred = model.predict(X_test) # предсказание
y_true = y_test # истина

print(f"MSE = {mean_squared_error(y_true, y_pred)}")
print(f"MAE = {mean_absolute_error(y_pred, y_true)}")


MSE = 1.402105331255363
MAE = 0.9037566472659067


Отклонение достаточно минимальное => модель хорошо предсказывает класс звезд.

Теперь используем градиентный бустинг.


In [7]:
# Разделение данных на обучающий и тестовый наборы
X = stars_for_lgb.drop(['Spectral Class'], axis=1)
y = stars_for_lgb['Spectral Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение модели LGBMRegressor
model = lgb.LGBMRegressor(num_leaves=31,  # Количество листьев в дереве
                          learning_rate=0.1,  # Скорость обучения
                          n_estimators=100,  # Количество деревьев
                        #   categorical_feature=starsFeatures,  # Список категориальных признаков
                          random_state=42)  # Задаем случайное начальное состояние для воспроизводимости


model.fit(X_train, y_train)

# Прогнозирование на тестовом наборе данных
y_pred = model.predict(X_test)

print('MSE = %s' % mean_squared_error(y_pred, y_test))

print("MAE = %s" % mean_absolute_error(y_pred, y_test))


MSE = 0.41206764572858084
MAE = 0.3437667383626974


Градиентный бустинг улучшил предсказание. Значения отклонений достаточно сильно приблизились к 0